# Homework 3 Anna Aksenova

Скачиваем и импортируем модули и библиотеки

In [1]:
!pip install nltk

In [2]:
!pip install spacy

In [3]:
!pip  install pyLDAvis

     |████████████████████████████████| 1.6MB 4.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=b580fca864a6834a9c0e0cbdce0288e67c474b97263297ed46f5c8b3a56f82b0
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [4]:
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

Скачиваем данные

In [7]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## Препроцессинг

In [8]:
# Convert to list
data = df.content.values.tolist()


# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [9]:
data[1]

'From: (Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu A fair number of brave souls who upgraded their SI clock oscillator have shared their experiences for this poll. Please send a brief message detailing your experiences with the procedure. Top speed attained, CPU rated speed, add on cards and adapters, heat sinks, hour of usage per day, floppy disk functionality with 800 and 1.4 m floppies are especially requested. I will be summarizing in the next two days, so please add to the network knowledge base if you have done the clock upgrade and havent answered this poll. Thanks. Guy Kuo '

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


## Работа с Маллет

In [15]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
 # update this path

--2020-10-17 13:04:25--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  6.46MB/s    in 2.4s    

2020-10-17 13:04:28 (6.46 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classi

In [16]:
import os
from gensim.models.wrappers import LdaMallet

os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet'

Запишем функцию для поиска лучшей модели. На вход она получит все данные для модели + минимальное число топиков, максимальное число топиков и шаг, с которым мы будем менять количество топиков в модели

In [17]:
def get_best_model(mallet_path, corpus, id2word, start, limit, step):
  best_model = None
  best_coh = 0
  for num_topics in tqdm(range(start, limit+1, step)):
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, 
                                                 num_topics=num_topics, id2word=id2word)

    # Compute Coherence Score
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    if coherence_ldamallet > best_coh:
      best_coh = coherence_ldamallet
      best_model = ldamallet
  return best_model, best_coh

In [18]:
ldamallet, coherence = get_best_model(mallet_path, corpus, id2word, 5, 25, 5)
print('\nCoherence:', coherence)

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
100%|██████████| 5/5 [10:04<00:00, 120.84s/it]


Coherence: 0.5377471629041074


In [19]:
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.000055  0.000274       1        1  5.666056
12     0.000024 -0.000163       2        1  5.643122
4      0.000861 -0.000091       3        1  5.448034
14    -0.000358  0.000410       4        1  5.030459
2      0.000137 -0.000426       5        1  4.870795
9      0.000137  0.000259       6        1  4.524920
7      0.000136 -0.000289       7        1  4.431594
19    -0.000329  0.000802       8        1  4.430501
5     -0.000086 -0.000035       9        1  4.360188
24    -0.000026  0.000387      10        1  4.318412
21    -0.000188 -0.000450      11        1  4.196332
3      0.000041 -0.000049      12        1  4.077455
6      0.000362 -0.000335      13        1  3.891178
11    -0.000209  0.000403      14        1  3.850334
15    -0.000365 -0.000050      15        1  3.790715
0     -0.000503 -0.000026      16        1  3.684335
18     0.000244  0.000367      17        1  3.655950
23    -0.000126  0.000231      18        1  3.645504
10     0.000890  0.000305      19        1  3.384308
16    -0.000478 -0.000526      20        1  3.230946
20     0.000050 -0.000154      21        1  3.186094
17     0.000249 -0.000175      22        1  2.969547
22     0.000027  0.000047      23        1  2.920299
1     -0.000174 -0.000596      24        1  2.461007
8     -0.000262 -0.000120      25        1  2.331913, topic_info=                       Term       Freq      Total Category  logprob  loglift
23392            marblehead  28.000000  28.000000  Default  30.0000  30.0000
33724  monosodium_glutamate  29.000000  29.000000  Default  29.0000  29.0000
13686          undercoating  28.000000  28.000000  Default  28.0000  28.0000
19427              scrubber  28.000000  28.000000  Default  27.0000  27.0000
4871                 impend  27.000000  27.000000  Default  26.0000  26.0000
...                     ...        ...        ...      ...      ...      ...
3340              hypocricy   0.891952  28.896186  Topic25 -10.2928   0.2804
35031                corona   0.887656  28.809175  Topic25 -10.2976   0.2786
20013             abduction   0.887210  29.307151  Topic25 -10.2981   0.2610
20999         deterioration   0.887231  29.514182  Topic25 -10.2981   0.2540
4341                squeeze   0.886025  29.375898  Topic25 -10.2995   0.2573

[1016 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
24324      1  0.069185    aam
24324      2  0.069185    aam
24324      3  0.069185    aam
24324      4  0.034593    aam
24324      5  0.034593    aam
...      ...       ...    ...
35124     21  0.033470  zubov
35124     22  0.033470  zubov
35124     23  0.033470  zubov
35124     24  0.033470  zubov
35124     25  0.033470  zubov

[24971 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 5, 15, 3, 10, 8, 20, 6, 25, 22, 4, 7, 12, 16, 1, 19, 24, 11, 17, 21, 18, 23, 2, 9])

### Подсчет весов слов

In [20]:
topics = ldamallet.show_topics(formatted=False, num_words=40) # 40 слов каждого топика с весами

In [21]:
# Создаем словарь для каждого топика вида "слово":вес 
# Добавляем в топик сет из характерных для него слов 
topd = {}
for i in topics:
  topd[i[0]] = [{}]
  words = {}
  for j in i[1]:
    topd[i[0]][0][j[0]] = j[1]
  topd[i[0]].append(set(topd[i[0]][0].keys()))

In [22]:
from collections import Counter

In [23]:
import random

In [24]:
top_text = {}
for num, text in enumerate(data_lemmatized):
  topics = Counter() # Счетчик сумм весов слов по топикам
  for word in text:
    for key in topd.keys():
      if word in topd[key][1]:
        topics[key] += topd[key][0][word]
  if topics: # Там есть короткие тексты, которые состоят из 6 слов и ни к чему не относятся
    top_text[num] = topics.most_common(1)[0][0]
  else:
    top_text[num] = random.randrange(0, 25, 1) # Выберем этим текстам случайную тему

In [25]:
# Создаем датафрейм с текстами и топиками
dat = {'texts': top_text.keys(), 'topics' : top_text.values()}
df = pd.DataFrame(data=dat)
df

,texts,topics
0,0,8
1,1,8
2,2,22
3,3,22
4,4,22
...,...,...
11309,11309,18
11310,11310,18
11311,11311,8
11312,11312,22


In [26]:
set(top_text.values()) # Как мы видим, не все "маленькие" топики могут быть "большими"

{0, 1, 2, 4, 7, 8, 10, 11, 12, 14, 17, 18, 21, 22, 23, 24}

### Tf-Idf

In [27]:
# Изменим словарь: каждому топику сопоставим тексты
texts_for_topics = {}
for key, value in top_text.items():
  if value not in texts_for_topics.keys():
    texts_for_topics[value] = []
  texts_for_topics[value].append(key)

In [28]:
# Добавим к id текста сам текст
# Словарь вида {топик : {id текста : текст, ...}, ...}
transformed = {}
for key, values in texts_for_topics.items():
  transformed[key] = {}
  for i in values:
    transformed[key][i] = data[i]

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
# Функция извлечения 5 слов с самым большим tf-idf
def top_tf_words(num, text):
  tf = X.transform([text])
  doc = 0
  feature_index = tf[doc,:].nonzero()[1]
  tfidf_scores = zip([feature_names[i] for i in feature_index],
                     [tf[0, x] for x in feature_index])
  words = [i[0] for i in Counter(tfidf_scores).most_common(5)]
  return num, words

In [32]:
# Проходимся по топикам и для каждого сначала считаем tf-idf
top_words = {}
for key, values in transformed.items():
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit(transformed[key].values())
  feature_names = vectorizer.get_feature_names()
  for num, text in values.items(): # Для текстов внутри топиков находим 5 слов
    num, words = top_tf_words(num, text)
    top_words[num] = words # Словарь {id текста : слова}

In [33]:
# Запишем слова в датафрейм
df['top-5'] = [top_words[text] for text in df['texts']]
df

,texts,topics,top-5
0,0,8,"[(your, 0.05800198795999559), (you, 0.08167709..."
1,1,8,"[(your, 0.0432143200322342), (you, 0.030426717..."
2,2,22,"[(you, 0.016225013010477428), (yea, 0.08088497..."
3,3,22,"[(you, 0.027863879428318274), (wrote, 0.063167..."
4,4,22,"[(you, 0.02439986409303975), (yet, 0.060977972..."
...,...,...,...
11309,11309,18,"[(zisfein, 0.18325644149588166), (your, 0.0201..."
11310,11310,18,"[(worked, 0.09286479208441915), (wires, 0.2714..."
11311,11311,8,"[(you, 0.02468506063450799), (yet, 0.054075291..."
11312,11312,22,"[(you, 0.019472156617598924), (wrong, 0.047698..."


### Coherence Score

Измеряет семантическую близость слов внутри одного топика, чтобы разделить "осознанные" темы от тем, которые выделились по статистическим неосмысленным совпадениям. 

Считается как медиана или среднее от попарной косинусной близости слов внутри топика.